In [1]:
#Dates for the periods

#Dates for the most recent period
start_date = '2023-01-01'
end_date = '2023-03-31'
#Dates for the period before the most recent period
start_date_last = '2022-10-01'
end_date_last = '2022-12-31'

#Schooling year
year_schooling = ['2022-2023', '2023-2024']

### Importing packages

In [2]:
#Importing packages

import pandas as pd
import numpy as np
from datetime import datetime
import pymysql
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
from dotenv import load_dotenv
import openpyxl

### Downloading Household children file from Commcare

In [3]:
#Connecting to Commcare
load_dotenv('id_cc.env')
email = os.getenv('EMAIL')
password_cc = os.getenv('PASSWORD')

#Defining the driver
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.implicitly_wait(1000)

#Creating login function
def commcare_login():
    driver.get(
        'https://www.commcarehq.org/a/caris-test/data/export/custom/new/case/download/f6ddce2133f8d233d9fbd9341220ed6f/'
    )
    driver.find_element_by_xpath('//*[@id="id_auth-username"]').send_keys(email)
    driver.find_element_by_xpath('//*[@id="id_auth-password"]').send_keys(password_cc)
    driver.find_element_by_css_selector('button[type=submit]').click()
       
commcare_login()

#Download the file: "Household child"
driver.find_element_by_xpath('//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element_by_xpath('//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()

#-------------------------------------------------------------------------------------------------------------------------

# Nouvelle forme appel OEV
def appel_oev():

    driver.get(
         'https://www.commcarehq.org/a/caris-test/data/export/custom/new/form/download/d24fc1a47af7508393152bf966a22f99/'
    )

appel_oev()

#Download the database "forme appel OEV"
driver.find_element_by_xpath('//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element_by_xpath('//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()

#-------------------------------------------------------------------------------------------------------------------------

#Positive kids Schooling Information
def positive_kids():
    driver.get(
    "https://www.commcarehq.org/a/caris-test/data/export/custom/new/case/download/67352311fba5d3a5e285283aa4951676/"
    )
    
positive_kids()

#Download positive database
driver.find_element_by_xpath('//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element_by_xpath('//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()


#-------------------------------------------------------------------------------------------------------------------------

#Gardening Information
def commcare_all_gardens():
    driver.get(
        'https://www.commcarehq.org/a/caris-test/data/export/custom/new/case/download/789629a97bddd10b4648d5138d17908e/'
    )

#All_gardens
commcare_all_gardens()

#Download the database "All gardens"
driver.find_element_by_xpath('//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element_by_xpath('//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()

True



====== WebDriver manager ======
Current google-chrome version is 111.0.5563
Get LATEST chromedriver version for 111.0.5563 google-chrome
Driver [C:\Users\Davidson Adrien\.wdm\drivers\chromedriver\win32\111.0.5563.64\chromedriver.exe] found in cache


In [30]:
# Import schooling positive data
school_positive = pd.read_excel(f"~/Downloads/schooling Enfant positif {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", parse_dates = True)
school_pos = school_positive[(school_positive['schooling_year'].isin(year_schooling)) &
                             (school_positive['eskew_peye'] == 'wi') & (school_positive['closed'] != True)]
school_pos = school_pos[['patient_code', 'fullname']]
school_pos.shape[0]

756

In [31]:
all_gardens = pd.read_excel(f"C:/Users/Davidson Adrien/Downloads/All Gardens {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", 
                            parse_dates = True)
all_gardens['info.last_modified_date'] = pd.to_datetime(all_gardens['info.last_modified_date'], errors = 'coerce')

#Creating new site and commune columns
all_gardens['site'] = all_gardens['caris_site'].str[:8]
all_gardens.loc[(all_gardens.site == 'ZLS/STMC'), 'site'] = 'ZLS/SSPE'
all_gardens.loc[(all_gardens.site == 'GRA/CSMN'), 'site'] = 'JER/CSMN'
all_gardens.loc[(all_gardens.site == 'ART/CSAR'), 'site'] = 'GON/CSAR'
all_gardens.loc[(all_gardens.site == 'GRA/HSPC'), 'site'] = 'JER/HSPC'
all_gardens.loc[(all_gardens.site == 'ZLS/POZS'), 'site'] = 'MON/POZS'
all_gardens.loc[(all_gardens.site == 'MAI/CSDM'), 'site'] = 'HIN/CSMD'

#Upper ST-code
all_gardens.rename(columns = {'patient_code': 'code_ST'}, inplace = True)
all_gardens['code_ST'] = all_gardens['code_ST'].str.upper()

#Create full column code
all_gardens['patient_code'] = all_gardens['site'] + '/' + all_gardens['code_ST']

pos_gardens = all_gardens[(all_gardens['beneficiary_type'] == 'Caris') & 
                          ((all_gardens['info.last_modified_date'] >= start_date_last) & 
                           (all_gardens['info.last_modified_date'] <= end_date)) & 
                          (all_gardens['closed'] != True)][['patient_code']]
pos_gardens.shape[0]

1081

In [32]:
#connecting to HIV Haiti
load_dotenv('dot.env')
user = os.getenv('MYSQL_USER')
password_hiv = os.getenv('MYSQL_PASSWORD')
host = os.getenv('MYSQL_HOST')
db = os.getenv('MYSQL_DB')

conn_text = f'mysql+pymysql://{user}:{password_hiv}@{host}/{db}'
engine = create_engine(conn_text)


True

### Connecting to HIV Haiti

In [33]:
sql = f'''

SELECT
patient.id as id_patient,
patient_code,
coalesce(ti.dob,tme.infant_dob,spy.date_of_birth) as dob,
coalesce(ti.gender,tme.infant_gender) as gender,
IF(arv.id_patient IS NOT NULL,
   'yes',
   'no') AS on_arv,
IF(statut.id_patient IS NOT NULL,
   'pos',
   'neg') AS statut,
IF(followup_q1 IS NOT NULL
   OR odk_q1 IS NOT NULL
   OR arv_q1 IS NOT NULL
   OR ques_q1 IS NOT NULL
   OR club_q1 IS NOT NULL
   OR mereenfant_q1 is not null
   OR specimen_q1 is not null
   OR rapid.rapid_q1 is not null
   OR opf_q1.id_patient is not null,
   'yes',
   'no') AS q1,
IF(followup_q1 IS NOT NULL, 'yes', 'no') AS followup_q1,
IF(odk_q1 IS NOT NULL, 'yes', 'no') AS odk_q1,
IF(arv_q1 IS NOT NULL, 'yes', 'no') AS arv_q1,
IF(ques_q1 IS NOT NULL, 'yes', 'no') AS ques_q1,
IF(club_q1 IS NOT NULL, 'yes', 'no') AS club_q1,
if(mereenfant_q1 is not null, 'yes','no') as mereenfant_q1,
if(specimen_q1 is not null, 'yes','no') as specimen_q1,
if(rapid.rapid_q1 is not null, 'yes','no') as rapid_q1,
IF(followup_q2 IS NOT NULL
   OR odk_q2 IS NOT NULL
   OR arv_q2 IS NOT NULL
   OR ques_q2 IS NOT NULL
   OR club_q2 IS NOT NULL
   or mereenfant_q2 is not null
   or specimen_q2 is not null
   OR rapid2.rapid_q2 is not null
   OR opf_q2.id_patient is not null,
   'yes',
   'no') AS q2,
IF(followup_q2 IS NOT NULL, 'yes', 'no') AS followup_q2,
IF(odk_q2 IS NOT NULL, 'yes', 'no') AS odk_q2,
IF(arv_q2 IS NOT NULL, 'yes', 'no') AS arv_q2,
IF(ques_q2 IS NOT NULL, 'yes', 'no') AS ques_q2,
IF(club_q2 IS NOT NULL, 'yes', 'no') AS club_q2,
if(mereenfant_q2 is not null, 'yes','no') as mereenfant_q2,
if(specimen_q2 is not null, 'yes','no') as specimen_q2,
if(rapid2.rapid_q2 is not null, 'yes','no') as rapid_q2,
if(opf_q1.id_patient is not null,'yes','no') as odk_phone_followup_q1,
if(opf_q2.id_patient is not null,'yes','no') as odk_phone_followup_q2,
patient.created_at,
lookup_hospital.name as hospital_name,
office,
IF(hn.id,'yes','no') as is_ugp,
if(ti.is_abandoned=1, 'yes', 'No') as Abandoned,
if(ti.is_dead=1, 'yes', 'No') as Dead

FROM
(SELECT
  *
    FROM
  (SELECT
    id_patient
    FROM
    tracking_followup
    WHERE
    date BETWEEN '{start_date_last}' AND '{end_date}' UNION
    
    ##### ADDING positive_pcr_1 or positive_pcr_2 START
    SELECT
    id_patient
    FROM
    tracking_infant
    WHERE
    (positive_pcr_1 BETWEEN '{start_date_last}' AND '{end_date}'
      OR positive_pcr_2 BETWEEN '{start_date_last}' AND '{end_date}')
    
    ##### ADDING positive_pcr_1 or positive_pcr_2 END
    
    ### ADDING ODK PHONE FOLLOWUP START
    UNION (
      select pt.id as id_patient from tracking_odk_phone_followup topf 
      left join patient pt on topf.patient_code = pt.patient_code
      where (topf.eccm_date between '{start_date_last}' AND '{end_date}')  and topf.name='Enquette Corona Enfant' and topf.eccm_joignable_par_tel!=0
    )
    UNION
    ### ADDING ODK PHONE FOLLOWUP END
    
    (SELECT DISTINCT
      patient.id AS id_patient
      FROM
      caris_db.odk_child_visit
      LEFT JOIN patient ON patient.patient_code = odk_child_visit.patient_code
      WHERE
      date_of_visit BETWEEN '{start_date_last}' AND '{end_date}'
      AND patient.id IS NOT NULL
      AND is_available_at_time_visit = 1
      AND linked_to_id_patient = 0) UNION SELECT
    id_patient
    FROM
    tracking_regime
    WHERE
    (start_date BETWEEN '{start_date_last}' AND '{end_date}' or end_date BETWEEN '{start_date_last}' AND '{end_date}')
    AND category = 'regime_infant_treatment' UNION SELECT
    id_patient
    FROM
    questionnaire_child
    WHERE
    date BETWEEN '{start_date_last}' AND '{end_date}' UNION SELECT
    id_patient
    FROM
    testing_mereenfant
    WHERE
    date BETWEEN '{start_date_last}' AND '{end_date}' UNION SELECT
    id_patient
    FROM
    testing_specimen
    WHERE
    date_blood_taken BETWEEN '{start_date_last}' AND '{end_date}'  UNION SELECT
    id_patient
    FROM
    questionnaire_proftraining18
    WHERE
    date BETWEEN '{start_date_last}' AND '{end_date}' UNION ((SELECT
                                                              id_patient
                                                              FROM
                                                              session
                                                              LEFT JOIN club_session ON club_session.id = id_club_session
                                                              LEFT JOIN club ON club_session.id_club = club.id
                                                              WHERE
                                                              is_present = 1
                                                              AND (date BETWEEN '{start_date}' AND '{end_date}'
                                                                   OR date BETWEEN '{start_date_last}' AND '{end_date_last}')
                                                              AND club_type != 1))) o) bot
LEFT JOIN
(SELECT DISTINCT
  id_patient AS followup_q1
  FROM
  tracking_followup
  WHERE
  date BETWEEN '{start_date_last}' AND '{end_date_last}') x ON followup_q1 = bot.id_patient
left join
(select distinct id_patient as specimen_q1 from testing_specimen WHERE
  date_blood_taken BETWEEN '{start_date_last}' AND '{end_date_last}') x2 ON specimen_q1 = bot.id_patient
left join
(select distinct id_patient as specimen_q2 from testing_specimen WHERE
  date_blood_taken BETWEEN '{start_date}' AND '{end_date}') x3 ON specimen_q2 = bot.id_patient
LEFT JOIN
(SELECT DISTINCT
  id_patient AS followup_q2
  FROM
  tracking_followup
  WHERE
  date BETWEEN '{start_date}' AND '{end_date}') y ON followup_q2 = bot.id_patient
LEFT JOIN
patient ON patient.id = bot.id_patient
LEFT JOIN
(SELECT DISTINCT
  id_patient AS mereenfant_q1
  FROM
  testing_mereenfant
  WHERE
  date BETWEEN '{start_date_last}' AND '{end_date_last}') me ON mereenfant_q1 = bot.id_patient
LEFT JOIN
(SELECT DISTINCT
  id_patient AS mereenfant_q2
  FROM
  testing_mereenfant
  WHERE
  date BETWEEN '{start_date}' AND '{end_date}') me2 ON mereenfant_q2 = bot.id_patient
LEFT JOIN
(SELECT DISTINCT
  patient.id AS odk_q1
  FROM
  caris_db.odk_child_visit
  LEFT JOIN patient ON patient.patient_code = odk_child_visit.patient_code
  WHERE
  date_of_visit BETWEEN '{start_date_last}' AND '{end_date_last}'
  AND patient.id IS NOT NULL
  AND is_available_at_time_visit = 1
  AND linked_to_id_patient = 0) z ON odk_q1 = bot.id_patient
LEFT JOIN
(SELECT DISTINCT
  patient.id AS odk_q2
  FROM
  caris_db.odk_child_visit
  LEFT JOIN patient ON patient.patient_code = odk_child_visit.patient_code
  WHERE
  date_of_visit BETWEEN '{start_date}' AND '{end_date}'
  AND patient.id IS NOT NULL
  AND is_available_at_time_visit = 1
  AND linked_to_id_patient = 0) zz ON odk_q2 = bot.id_patient
LEFT JOIN
(SELECT DISTINCT
  id_patient AS arv_q2
  FROM
  tracking_regime
  WHERE
  start_date BETWEEN '{start_date}' AND '{end_date}' or end_date BETWEEN '{start_date}' AND '{end_date}') a2 ON arv_q2 = bot.id_patient
LEFT JOIN
(SELECT DISTINCT
  id_patient AS arv_q1
  FROM
  tracking_regime
  WHERE
  start_date BETWEEN '{start_date_last}' AND '{end_date_last}' or end_date BETWEEN '{start_date_last}' AND '{end_date_last}') a1 ON arv_q1 = bot.id_patient
LEFT JOIN
(SELECT
  id_patient AS ques_q1
  FROM
  questionnaire_child
  WHERE
  date BETWEEN '{start_date_last}' AND '{end_date_last}' UNION SELECT
  id_patient AS ques_q1
  FROM
  questionnaire_proftraining18
  WHERE
  date BETWEEN '{start_date_last}' AND '{end_date_last}') q1 ON ques_q1 = bot.id_patient
LEFT JOIN
(SELECT
  id_patient AS ques_q2
  FROM
  questionnaire_child
  WHERE
  date BETWEEN '{start_date}' AND '{end_date}' UNION SELECT
  id_patient AS ques_q2
  FROM
  questionnaire_proftraining18
  WHERE
  date BETWEEN '{start_date}' AND '{end_date}') q2 ON ques_q2 = bot.id_patient
LEFT JOIN
(SELECT DISTINCT
  id_patient AS club_q1
  FROM
  session
  LEFT JOIN club_session ON club_session.id = id_club_session
  LEFT JOIN club ON club_session.id_club = club.id
  WHERE
  is_present = 1
  AND date BETWEEN '{start_date_last}' AND '{end_date_last}'
  AND club_type != 1) c1 ON club_q1 = bot.id_patient
LEFT JOIN
(SELECT DISTINCT
  id_patient AS club_q2
  FROM
  session
  LEFT JOIN club_session ON club_session.id = id_club_session
  LEFT JOIN club ON club_session.id_club = club.id
  WHERE
  is_present = 1
  AND date BETWEEN '{start_date}' AND '{end_date}'
  AND club_type != 1) c2 ON club_q2 = bot.id_patient
LEFT JOIN
lookup_hospital ON CONCAT(lookup_hospital.city_code,
                          '/',
                          lookup_hospital.hospital_code) = LEFT(patient_code, 8)
LEFT JOIN
(SELECT DISTINCT
  id_patient
  FROM
  tracking_regime
  WHERE
  category = 'regime_infant_treatment') arv ON arv.id_patient = bot.id_patient
##### ADDING positive_pcr_1 or positive_pcr_2 START
LEFT JOIN (SELECT
           id_patient as rapid_q1
           FROM
           tracking_infant
           WHERE
           (positive_pcr_1 BETWEEN '{start_date_last}' AND '{end_date_last}'
             OR positive_pcr_2 BETWEEN '{start_date_last}' AND '{end_date_last}')) rapid on rapid.rapid_q1=bot.id_patient

##### ADDING positive_pcr_1 or positive_pcr_2 END

##### ADDING positive_pcr_1 or positive_pcr_2 START
LEFT JOIN (SELECT
           id_patient as rapid_q2
           FROM
           tracking_infant
           WHERE
           (positive_pcr_1 BETWEEN '{start_date}' AND '{end_date}'
             OR positive_pcr_2 BETWEEN '{start_date}' AND '{end_date}')) rapid2 on rapid2.rapid_q2=bot.id_patient

##### ADDING positive_pcr_1 or positive_pcr_2 END

### ODK PHONE FOLLOWUP START

LEFT JOIN (
  select case_id,eccm_date, pt3.id as id_patient from tracking_odk_phone_followup topf
  left join patient pt3 on  pt3.patient_code = topf.patient_code
  where topf.eccm_joignable_par_tel!=0 and topf.eccm_date BETWEEN '{start_date_last}' AND '{end_date_last}' group by topf.case_id
) opf_q1 on opf_q1.id_patient= bot.id_patient
LEFT JOIN (
  select case_id,eccm_date, pt4.id as id_patient from tracking_odk_phone_followup topf
  left join caris_db.odk_hivhaiti_backward_compatibility obc on obc.odk_case_id=topf.case_id
  left join patient pt4 on pt4.patient_code = topf.patient_code
  where topf.eccm_joignable_par_tel!=0 and topf.eccm_date BETWEEN '{start_date}' AND '{end_date}' group by topf.case_id
) opf_q2 on opf_q2.id_patient= bot.id_patient
### ODK PHONE FOLLOWUP END
LEFT JOIN
(SELECT
  id_patient
  FROM
  tracking_followup UNION SELECT
  id_patient
  FROM
  tracking_regime
  WHERE
  category = 'regime_infant_treatment' UNION SELECT
  id_patient
  FROM
  club_patient UNION SELECT
  id_patient
  FROM
  questionnaire_child UNION SELECT
  id_patient
  FROM
  questionnaire_proftraining18 UNION SELECT
  id_patient
  FROM
  testing_specimen
  WHERE
  pcr_result = 1 UNION SELECT
  id_patient
  FROM
  testing_result
  WHERE
  result = 1 UNION SELECT
  id_patient
  FROM
  tracking_infant
  WHERE
  positive_pcr_1 != '0000-00-00'
  OR positive_pcr_2 != '0000-00-00') statut ON statut.id_patient = bot.id_patient

left join tracking_infant ti on ti.id_patient=bot.id_patient
left join testing_mereenfant tme on tme.id_patient=bot.id_patient
left join (select max(spx.date_of_birth) as date_of_birth, spx.id_patient from testing_specimen spx group by spx.id_patient) spy on spy.id_patient=bot.id_patient
left join hospital_network hn on (hn.id_hospital=lookup_hospital.id and id_network=7)

WHERE
linked_to_id_patient = 0

'''

#Reading the OVC Helper Mother file
data = pd.read_sql_query(sql, engine, parse_dates = True)
data.shape[0]


3901

In [6]:
# data.to_excel('OEV_servis_Q2FY23_02222023.xlsx', index = True)

### Preparing OVC data

In [34]:

#Importing dataset with hospital name, department, commune, networks name
infos = pd.read_excel('Caris Sites_modified.xlsx', usecols = ['CODE LABO', 'HOPITAL', 'RESEAU','DEPT', 'COMMUNE'],sheet_name = 'Caris Sites')
infos.rename(columns = {'CODE LABO': 'site',
                       'HOPITAL': 'hospital',
                       'DEPT': 'Department',
                       'COMMUNE': 'Commune',
                       'RESEAU': 'network'}, inplace = True)

#Creating the sites column to merge with the other datasets
data['site'] = data['patient_code'].apply(lambda x: x[0:8])

#converting dob column into datetime type
data['dob'] = pd.to_datetime(data['dob'], errors = 'coerce')

#Adding some required columns
data_merge = data.merge(infos, on = 'site', how = 'left')
data_on_hivhaiti = data_merge

#filtering for patient on HIV Haiti and not belonging to UGP
# data_on_hivhaiti=data_merge[data_merge['Dead']=='No']
# data_on_hivhaiti=data_on_hivhaiti[data_on_hivhaiti['Abandoned']=='No']

#Creating age column
data_on_hivhaiti['dob'].fillna('1901-01-01', inplace = True)
#data_on_hivhaiti.loc[(data_on_hivhaiti.dob == '1775-01-01'), 'dob'] = '1901-01-01'


#Transforming dob into datetime type
data_on_hivhaiti['dob'] = pd.to_datetime(data_on_hivhaiti['dob'], errors = 'coerce')

#Creating the age column
def get_age(date):
        return relativedelta(parse(end_date), date).years

data_on_hivhaiti['age'] = data_on_hivhaiti['dob'].map(get_age)

#OVC >= 18
data_sup18 = data_on_hivhaiti[(data_on_hivhaiti['age'] >= 18) & 
                           ((data_on_hivhaiti['patient_code'].isin(school_pos['patient_code'])) | 
                           (data_on_hivhaiti['patient_code'].isin(pos_gardens['patient_code'])))]

#OVC < 18
data_inf18 = data_on_hivhaiti[data_on_hivhaiti['age'] < 18]

#All OVC data
ovc_data = pd.concat([data_inf18, data_sup18])

#ovc_data.head(5)
ovc_data.shape

(3361, 38)

In [35]:
#new calls file
appel = pd.read_excel(f"C:/Users/Davidson Adrien/Downloads/Caris Health Agent - Enfant - APPELS OEV (created 2022-03-28) {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx")
appel.rename(columns = {'form.appels_oev.date_appel': 'date_appel',
                       'form.appels_oev.parenttuteur_trouve': 'found',
                       'form.appels_oev.patient_code': 'patient_code'}, inplace = True)
appel['date_appel'] = pd.to_datetime(appel['date_appel'], errors = 'coerce')
appel['found'] = pd.to_numeric(appel['found'], errors = 'coerce')

appel_qi = appel[(appel['date_appel'] >= start_date_last) & (appel['date_appel'] <= end_date_last) &
                 (appel['found'] == 1)][['patient_code', 'date_appel', 'found']].drop_duplicates('patient_code')
appel_qj = appel[(appel['date_appel'] >= start_date) & (appel['date_appel'] <= end_date) & 
                 (appel['found']) == 1][['patient_code', 'date_appel', 'found']].drop_duplicates('patient_code')

#====================================================================================================
#create table alike services table
all_appel_served_q1q2 = appel_qi.merge(appel_qj[['patient_code', 'date_appel']], on = 'patient_code', how = 'left')

#====================================================================================================

print(f"Nombre d'appels Qi: {appel_qi.shape[0]}")
print(f"Nombre d'appels Qj: {appel_qj.shape[0]}")
print(f"Servis en Q1 et Q2: {all_appel_served_q1q2[all_appel_served_q1q2['date_appel_y'].notnull()].shape[0]}")

Nombre d'appels Qi: 987
Nombre d'appels Qj: 809
Servis en Q1 et Q2: 509


In [36]:
# create new columns to receive calls information
ovc_data['odk_call_q1'] = "no"
ovc_data["odk_call_q2"] = "no"

# make changes in services according to calls received by beneficiaries
for code in appel_qi['patient_code']:
    ovc_data.loc[(ovc_data['patient_code'] == code), 'odk_call_q1'] = 'yes'

for code in appel_qj['patient_code']:
    ovc_data.loc[(ovc_data['patient_code'] == code), 'odk_call_q2'] = 'yes'

ovc_data.loc[(ovc_data['odk_call_q1'] == 'yes'), 'q1'] = 'yes'
ovc_data.loc[(ovc_data['odk_call_q2'] == 'yes'), 'q2'] = 'yes'

In [23]:
#ovc_data.to_excel('Treated_OEV_servis_Q2FY23.xlsx', index = False)

In [24]:
#data_on_hivhaiti.to_excel('All_OEV_Served_FY23Q22.xlsx', index = False)

In [53]:
#Served children for the 1st semester

#-------------------------------------------------------------------------------------------------------------------------

#served children in Q1 and Q2
q1q2 = ovc_data[(ovc_data.q1 == 'yes') & (ovc_data.q2 == 'yes') & (ovc_data.network!='UGP')][['patient_code', 'dob','gender','club_q1', 'club_q2', 'on_arv', 'Commune', 'age']]

#served children created while Q2
q2 = ovc_data[(ovc_data.created_at >= start_date) & (ovc_data.created_at <= end_date) & (ovc_data.network!='UGP')][['patient_code',  'dob','gender','club_q1', 'club_q2', 'on_arv', 'Commune', 'age']]

#ugp-served children (specimen or mereenfant) in q1 and q2
ugp_data=ovc_data[ovc_data['network'] == 'UGP']
# ugp_data = ugp_data[ugp_data['is_ugp'] == 'yes']
ugp_q1q2=ugp_data[((ugp_data['mereenfant_q1']=='yes') | (ugp_data['specimen_q1']=='yes')) & ((ugp_data['mereenfant_q2']=='yes') |
                    (ugp_data['specimen_q2']=='yes'))][['patient_code', 'dob','gender', 'club_q1', 'club_q2', 'on_arv', 'Commune', 'age']]

#ugp-served children (specimen or mereenfant) in q2
ugp_q2=ugp_data[(ovc_data.created_at >= start_date) & (ovc_data.created_at <= end_date)]
ugp_q2=ugp_q2[(ugp_q2['mereenfant_q2']=='yes') | (ugp_q2['specimen_q2']=='yes')][['patient_code',  'dob','gender', 'club_q1', 'club_q2', 'on_arv', 'Commune', 'age']]


#Concatenation of the two previous datasets
OVC1 = pd.concat([q1q2, q2,ugp_q1q2,ugp_q2]).drop_duplicates('patient_code').reset_index(drop = True)
print(f"\nNumber of children (OVC-)served for the semester: {OVC1.shape[0]}.")
OVC1.head()


Number of children (OVC-)served for the semester: 2111.


,patient_code,dob,gender,club_q1,club_q2,on_arv,Commune,age
0,PAP/CHAP/ST02964,2016-09-29,2.0,no,no,yes,Carrefour,6
1,PAP/CHAP/ST02133,2005-12-24,1.0,no,no,yes,Carrefour,17
2,PAP/CHAP/ST02722,2017-07-01,2.0,no,no,yes,Carrefour,5
3,PAP/CHAP/ST02182,2012-05-11,2.0,yes,yes,yes,Carrefour,10
4,PAP/CHAP/ST02816,2015-10-20,1.0,no,no,yes,Carrefour,7


In [26]:
OVC1.to_excel('direct_benef_all_ages_FY23Q22.xlsx', index = False)

# Direct beneficiaries

In [56]:
# List of direct beneficiaries
#List of children in club for the semester
club_child = OVC1[(OVC1.club_q1 == 'yes') | (OVC1.club_q2 == 'yes')][['patient_code','dob','gender','Commune']]
club_child

,patient_code,dob,gender,Commune
3,PAP/CHAP/ST02182,2012-05-11,2.0,Carrefour
7,PAP/CHAP/ST01079,2008-01-09,1.0,Carrefour
8,PAP/CHAP/ST01622,2006-10-10,2.0,Carrefour
9,PAP/CHAP/ST00895,2006-04-22,1.0,Carrefour
10,PAP/CHAP/ST03199,2010-07-28,1.0,Carrefour
...,...,...,...,...
1909,LIM/HOSJ/ST03242,2008-04-20,0.0,Limbe
1917,PAP/CMSP/ST01747,2007-12-16,1.0,Delmas
1918,PAP/CMSP/ST01749,2007-03-07,2.0,Delmas
2008,PIG/CSSR/ST00883,2008-01-17,1.0,Saint-Raphael


In [39]:
club_child.to_excel("Child_in_club_Q2FY23_03162022.xlsx", index = False)

In [57]:
#Importing household child file
hh_child = pd.read_excel(f"~/Downloads/household_child  {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", sheet_name = 'Cases', parse_dates = True)

hh_child.full_code_patient_menage = hh_child.full_code_patient_menage.str.upper()

#Importing All patient table from HIV Haiti to know whether the children code is on HIV Haiti or not
sql2 = 'select * from patient'
all_patient = pd.read_sql_query(sql2, engine, parse_dates = True)
all_patient.shape[0]

172009

In [58]:
#Regime ARV all patient
sql3 = '''
SELECT 
       p.patient_code, id_patient,
           MIN(IF(start_date != '0000-00-00', start_date, NULL)) AS start_date,
           GROUP_CONCAT(la.acronym
               SEPARATOR '-') AS regime,
           GROUP_CONCAT(tr.start_date
               SEPARATOR ' | ') AS all_start_date,
           GROUP_CONCAT(tr.end_date
               SEPARATOR ' | ') AS all_end_date
   FROM
       tracking_regime tr
	LEFT JOIN patient p ON p.id = tr.id_patient
   LEFT JOIN lookup_arv la ON la.id = tr.id_arv
   WHERE
    category = 'regime_infant_treatment' and tr.end_date='0000-00-00'
   GROUP BY id_patient
'''

regime = pd.read_sql_query(sql3, engine, parse_dates = True)
regime.rename(columns = {'patient_code': 'full_code_patient_menage',
                        'start_date': 'arv_start_date'}, 
              inplace = True)

regime = pd.read_sql_query(sql3, engine, parse_dates = True)
regime.rename(columns = {'patient_code': 'full_code_patient_menage',
                        'start_date': 'arv_start_date'}, 
               inplace = True)

In [59]:
#Function to get the age range
def age_range(age):
    if age < 1:
        return '< 1'
    elif age in range(1, 5):
        return '1 - 4'
    elif age in range(5, 10):
        return '5 - 9'
    elif age in range(10, 15):
        return '10 - 14'
    elif age in range(15, 18):
        return '15 - 17'
    else:
        return '18+'

#----------------------------------------------------------------------------------------------------------------------
#Function to know whether a patient code is on HIV Haiti or not
def on_hivhaiti(code):
    if code.upper() in all_patient.patient_code.tolist():
        return 'yes'
    else:
        return 'no'
    
#----------------------------------------------------------------------------------------------------------------------

#
hh_child.rename(columns = {'main_infant_code':'patient_code'}, 
              inplace = True)

#data for only child in club
hhm_club = hh_child[hh_child['patient_code'].str.lower().isin(club_child['patient_code'].str.lower())]
hhm_club['patient_code'] = hhm_club['patient_code'].str.upper()

#preparing age columns
hhm_club.loc[(hhm_club.age_in_year == '---'), 'age_in_year'] = ''
hhm_club['age_in_year'] = pd.to_numeric(hhm_club['age_in_year'])
hhm_club['age_range'] = hhm_club['age_in_year'].map(age_range)

#column specifying the code is on HIV Haiti
hhm_club['on_HIVHAITI'] = hhm_club['full_code_patient_menage'].map(on_hivhaiti)

#Replacing code by true gender values
hhm_club.loc[(hhm_club.gender == 1), 'gender'] = 'Male'
hhm_club.loc[(hhm_club.gender == 2), 'gender'] = 'Female'


In [60]:
#Adding the communes column
hhm_club = hhm_club.merge(OVC1[['patient_code', 'Commune']], on = 'patient_code', how = 'left')
hhm_club.rename(columns = {'full_code_patient_menage': 'patient_code2'}, inplace = True)
OVC1.rename(columns = {'patient_code': 'patient_code2'}, inplace = True)
hhm_club = hhm_club.merge(OVC1[['patient_code2', 'on_arv']], on = 'patient_code2', how = 'left')
hhm_club['on_arv'].fillna('---', inplace = True)
hhm_club.head(2)
hhm_club.shape

,number,caseid,cty,dob,first_name,gender,hiv_test,hiv_test_date,hiv_test_result,hospital_code,...,code_incorrect_raison,confirmation_code_by_agent,consent,has_schooling_form_2021,school_name,Full_code_patient_menage1,age_range,on_HIVHAITI,Commune,on_arv
0,0,5cdc6db5-3642-4cd2-aa2f-f4c2acf6a5b0,---,1970-08-09,wisly,Male,---,---,---,---,...,---,---,---,---,---,---,18+,no,Dessalines,---
1,71,c72996b5-ce37-44be-a718-7d365dbc5207,---,1989-07-05,Acao,Male,---,---,---,---,...,---,---,---,---,---,---,18+,no,Jeremie,---


(5305, 46)

In [61]:
hhm_club.to_excel('Comptage_OEV_FY223Q2_03162023.xlsx', index = False)

In [24]:
# hhm_club.groupby('patient_code').count()

In [62]:
#creating the siblings dataset and the dataset for the other kids
hhm_ff = hhm_club[hhm_club.infant_relationship == str(3)]
hhm_other = hhm_club[hhm_club.infant_relationship != str(3)]
hhm_other.loc[(hhm_other['name'] == 'Metellus Rose Derlove'), 'gender'] = 'Female'


#Size
hhm_ff_stat = hhm_club[(hhm_club.infant_relationship == str(3)) & (hhm_club.age_range != '18+')]
hhm_other_stat = hhm_club[(hhm_club.infant_relationship != str(3)) & (hhm_club.age_range != '18+')]
hhm_ff_stat.shape

hhm_other_stat.loc[(hhm_other_stat['name'] == 'Metellus Rose Derlove'), 'gender'] = 'Female'
hhm_other_stat.shape

(1831, 46)

(803, 46)

In [63]:
hhm_ff_stat.to_excel('siblings_FY23Q2_03162023.xlsx', index = False)
hhm_other_stat.to_excel('other_kids_FY23Q2_03162023.xlsx', index = False)

# DATIM Direct Siblings

In [65]:
#Creating Datim Table for siblings' kids
index_ff = hhm_ff.pivot_table(index = 'Commune', columns = ['gender', 'age_range'], values = 'patient_code', 
                              aggfunc = len, fill_value = 0, margins = True, margins_name = 'Total')
columns = [('Female', '< 1'),
 ('Female', '1 - 4'),
 ('Female', '5 - 9'),
 ('Female', '10 - 14'),
 ('Female', '15 - 17'),
 ('Female', '18+'),
 ('Male', '< 1'),
 ('Male', '1 - 4'),
 ('Male', '5 - 9'),
 ('Male', '10 - 14'),
 ('Male', '15 - 17'),
 ('Male', '18+')]
index_ff = index_ff.reindex(columns = columns)
# index_ff[('Female', '18+')] = 0
# index_ff[('Male', '18+')] = 0
index_ff.reset_index(inplace = True)
index_ff

gender                 Commune Female                                  Male  \
age_range                         < 1 1 - 4 5 - 9 10 - 14 15 - 17  18+  < 1   
0              Anse D'Hainault      0     0     3       2       1    1    0   
1                Anse-a-Foleur      0     0     0       1       0    0    0   
2                Anse-a-Galets      0     2     2       5       3   10    0   
3                  Anse-a-Veau      0     0     1       0       0    0    0   
4                        Aquin      0     1     1       5       3    6    0   
5                     Arcahaie      1     4     2       2       6    6    0   
6                  Cap-Haitien      1     3     2       3       1    0    1   
7                    Capotille      0     0     0       0       0    0    0   
8                       Carice      0     0     1       0       0    0    0   
9                    Carrefour      0     8     7      10       2    4    0   
10                   Cavaillon      1     1     1       0       0    1    1   
11                       Cayes      1     0     0       2       1    0    1   
12          Croix-des-Bouquets      0     2    26      36       9   13    1   
13                      Delmas      1    16    53      64      19   32    0   
14                  Dessalines      0     2     5       3       4    4    1   
15                      Ennery      0     0     2       2       0    2    1   
16                    Gonaives      1     4     9       7       4    9    0   
17                  Gros-Morne      1     2     9      13       3   12    5   
18                      Hinche      0     0     1       0       0    0    0   
19                  Jean-Rabel      0     0     1       0       0    0    0   
20                     Jeremie      0     2     1       1       0    0    0   
21                    Kenscoff      1     1     4       1       0    4    0   
22                     L'Asile      0     1     1       0       0    1    0   
23                   La Tortue      0     1     0       0       0    0    0   
24                     Leogane      0     3     0       2       1    3    0   
25                       Limbe      0     1     7       3       4    5    1   
26                    Limonade      0     0     0       0       0    2    0   
27                   Marmelade      0     3     1       6       0    5    1   
28                       Milot      0     4     3       8      11   20    3   
29                   Miragoane      0     4     4       9       3   13    0   
30               Mombin-Crochu      0     0     1       1       0    0    0   
31               Mont-Organise      0     2     7       8       1    1    0   
32                 Ouanaminthe      1     5     0       5       3    0    0   
33                 Petit-Goave      0     2     6       4       7    9    0   
34                  Petit-Trou      0     0     0       1       0    0    0   
35                      Pignon      0     3     6       8       3    7    0   
36                      Pilate      2     3     8      18       5    9    0   
37              Plaine-du-Nord      0     0     0       3       2    1    0   
38                   Plaisance      0     0     1       4       0    1    0   
39                 Port-Margot      0     0     1       1       4    1    0   
40              Port-au-Prince      0     5    27      32      11    9    0   
41                Port-de-Paix      6     6    12      27      16   18    0   
42         Saint-Louis-du-Nord      0     0     1       0       0    2    0   
43                  Saint-Marc      0     0     1       0       2    3    0   
44               Saint-Raphael      0     2     2       1       1    2    0   
45                     Tabarre      3    14    30      63      41   54    5   
46               Terrier-Rouge      3     1     7       4       2    3    2   
47                Trou-du-Nord      0     2     4       2       0    2    1   
48                       Total     23   110   261 

## OVC_HIVSTAT _ OEV Siblings

In [17]:
#Negative children
# # hhm_negative_children = hhm_ff_stat[(hhm_ff_stat.hiv_test_result == 'negatif') & (hhm_ff_stat.age_in_year <= 17)]
# hhm_negative_children = hhm_ff_stat_stat[hhm_ff_stat_stat.hiv_test_result == 'negatif']
# negative_children = hhm_negative_children.pivot_table(index = 'Commune', values = 'patient_code', 
#                                                       columns = 'hiv_test_result', aggfunc = len, fill_value = 0)
# negative_children.rename(columns = {'negatif': 'negative'}, inplace = True)
# negative_children.reset_index(inplace = True)
# negative_children.head()



#==========================================================================================================================

hhm_negative_children = hhm_ff_stat[(hhm_ff_stat.hiv_test_result == 'negatif') & (hhm_ff_stat.age_in_year <= 17)]
negative_children = hhm_negative_children.pivot_table(index = 'Commune', values = 'patient_code', 
                                                      columns = ['hiv_test_result', 'gender','age_range'], aggfunc = len, fill_value = 0)
negative_children.rename(columns = {'negatif': 'negative'}, inplace = True)

columns = [('negative','Female', '< 1'),
 ('negative','Female', '1 - 4'),
 ('negative','Female', '5 - 9'),
 ('negative','Female', '10 - 14'),
 ('negative','Female', '15 - 17'),
 ('negative','Male', '< 1'),
 ('negative','Male', '1 - 4'),
 ('negative','Male', '5 - 9'),
 ('negative','Male', '10 - 14'),
 ('negative','Male', '15 - 17')]

negative_children = negative_children.reindex(columns = columns)
negative_children.columns = ['_'.join(col) for col in negative_children.columns.values]
negative_children.reset_index(inplace = True)

negative_children.head()

,Commune,negative_Female_< 1,negative_Female_1 - 4,negative_Female_5 - 9,negative_Female_10 - 14,negative_Female_15 - 17,negative_Male_< 1,negative_Male_1 - 4,negative_Male_5 - 9,negative_Male_10 - 14,negative_Male_15 - 17
0,Anse D'Hainault,0,0,3,1,1,0,2,0,0,0
1,Anse-a-Foleur,0,0,0,0,0,0,0,1,0,0
2,Anse-a-Galets,0,1,3,3,2,0,0,5,3,4
3,Anse-a-Veau,0,0,1,0,0,0,2,0,0,0
4,Aquin,0,1,1,3,4,0,2,2,5,0


In [82]:
hhm_negative_children.shape[0]

1287

In [191]:
#hhm_positive_children.on_arv.unique()

In [18]:
#Positive children
def on_arv(date):
    if pd.notna(date):
        return 'yes'
    else:
        return 'no'


#hhm_positive_children = hhm_ff[(hhm_ff.hiv_test_result == 'positif') & (hhm_ff.age_in_year <= 17) & (hhm_ff.on_HIVHAITI == 'yes')].drop_duplicates('patient_code2')
hhm_positive_children = hhm_ff_stat[(hhm_ff_stat.hiv_test_result == 'positif') & (hhm_ff.on_HIVHAITI == 'yes')].drop_duplicates('patient_code2')
positive_children = hhm_positive_children.pivot_table(values = 'patient_code', index = 'Commune', columns = ['hiv_test_result', 'gender','age_range'],
                                                     aggfunc = len, fill_value = 0)
positive_children.rename(columns = {'positif': 'positive'}, inplace = True)

columns = [('positive','Female', '< 1'),
 ('positive','Female', '1 - 4'),
 ('positive','Female', '5 - 9'),
 ('positive','Female', '10 - 14'),
 ('positive','Female', '15 - 17'),
 ('positive','Male', '< 1'),
 ('positive','Male', '1 - 4'),
 ('positive','Male', '5 - 9'),
 ('positive','Male', '10 - 14'),
 ('positive','Male', '15 - 17')]

positive_children = positive_children.reindex(columns = columns)
positive_children.fillna(0, inplace = True)
positive_children.columns = ['_'.join(col) for col in positive_children.columns.values]


positive_children.reset_index(inplace = True)
positive_children.head()


,Commune,positive_Female_< 1,positive_Female_1 - 4,positive_Female_5 - 9,positive_Female_10 - 14,positive_Female_15 - 17,positive_Male_< 1,positive_Male_1 - 4,positive_Male_5 - 9,positive_Male_10 - 14,positive_Male_15 - 17
0,Delmas,0.0,0.0,1,0.0,0,0.0,0.0,0.0,0.0,0.0
1,Gros-Morne,0.0,0.0,0,0.0,1,0.0,0.0,0.0,0.0,0.0


In [19]:
hhm_positive_children[['patient_code2','patient_code', 'on_arv']]

,patient_code2,patient_code,on_arv
146,GRO/HAMG/00447,GRO/HAMG/ST00938,---
4165,PAP/CMSP/ST01602,PAP/CMSP/ST01563,yes


In [194]:
# hhm_positive_children.to_excel('positive_siblings.xlsx')

In [20]:
#Dataset children on arv
hhm_children_on_arv = hhm_positive_children[hhm_positive_children.on_arv == 'yes']

children_on_arv = hhm_children_on_arv.pivot_table(values = 'patient_code', index = 'Commune', 
                                                  columns = ['on_arv', 'gender','age_range'], aggfunc = len, fill_value = 0)

children_on_arv.rename(columns = {'yes': 'on_arv'}, inplace = True)

columns = [('on_arv','Female', '< 1'),
 ('on_arv','Female', '1 - 4'),
 ('on_arv','Female', '5 - 9'),
 ('on_arv','Female', '10 - 14'),
 ('on_arv','Female', '15 - 17'),
 ('on_arv','Male', '< 1'),
 ('on_arv','Male', '1 - 4'),
 ('on_arv','Male', '5 - 9'),
 ('on_arv','Male', '10 - 14'),
 ('on_arv','Male', '15 - 17')]
children_on_arv = children_on_arv.reindex(columns = columns)
children_on_arv.columns = ['_'.join(col) for col in children_on_arv.columns.values]
children_on_arv.fillna(0, inplace = True)
children_on_arv.reset_index(inplace = True)
children_on_arv

#Dataset children not on arv
hhm_children_not_on_arv = hhm_positive_children[hhm_positive_children.on_arv == '---']

children_not_on_arv = hhm_children_not_on_arv.pivot_table(values = 'patient_code', index = 'Commune',
                                                         columns = ['on_arv','gender','age_range'], aggfunc = len, fill_value = 0)

children_not_on_arv.rename(columns = {'---': 'not_on_arv'}, inplace = True)

columns = [('not_on_arv','Female', '< 1'),
 ('not_on_arv','Female', '1 - 4'),
 ('not_on_arv','Female', '5 - 9'),
 ('not_on_arv','Female', '10 - 14'),
 ('not_on_arv','Female', '15 - 17'),
 ('not_on_arv','Male', '< 1'),
 ('not_on_arv','Male', '1 - 4'),
 ('not_on_arv','Male', '5 - 9'),
 ('not_on_arv','Male', '10 - 14'),
 ('not_on_arv','Male', '15 - 17')]
children_not_on_arv = children_not_on_arv.reindex(columns = columns)
children_not_on_arv.columns = ['_'.join(col) for col in children_not_on_arv.columns.values]
children_not_on_arv.fillna(0, inplace = True)
children_not_on_arv.reset_index(inplace = True)
children_not_on_arv

,Commune,on_arv_Female_< 1,on_arv_Female_1 - 4,on_arv_Female_5 - 9,on_arv_Female_10 - 14,on_arv_Female_15 - 17,on_arv_Male_< 1,on_arv_Male_1 - 4,on_arv_Male_5 - 9,on_arv_Male_10 - 14,on_arv_Male_15 - 17
0,Delmas,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,Commune,not_on_arv_Female_< 1,not_on_arv_Female_1 - 4,not_on_arv_Female_5 - 9,not_on_arv_Female_10 - 14,not_on_arv_Female_15 - 17,not_on_arv_Male_< 1,not_on_arv_Male_1 - 4,not_on_arv_Male_5 - 9,not_on_arv_Male_10 - 14,not_on_arv_Male_15 - 17
0,Gros-Morne,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0


### No HIV reported (Status Unknown)

In [21]:
#Positive children not on HIV Haiti
hhm_positive_children_not_on_hivhaiti = hhm_ff_stat[(hhm_ff_stat.hiv_test_result == 'positif') & (hhm_ff_stat.on_HIVHAITI == 'no')]

positive_children_not_on_hivhaiti = hhm_positive_children_not_on_hivhaiti.pivot_table(values = 'patient_code', index = 'Commune',
                                                                                     columns = ['hiv_test_result', 'gender','age_range'], aggfunc = len,
                                                                                     fill_value = 0)
positive_children_not_on_hivhaiti.rename(columns = {'positif': 'No_HIV_reported_Status_Unknown'}, inplace = True)

columns = [('No_HIV_reported_Status_Unknown','Female', '< 1'),
 ('No_HIV_reported_Status_Unknown','Female', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Female', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Female', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Female', '15 - 17'),
 ('No_HIV_reported_Status_Unknown','Male', '< 1'),
 ('No_HIV_reported_Status_Unknown','Male', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Male', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Male', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Male', '15 - 17')]
positive_children_not_on_hivhaiti = positive_children_not_on_hivhaiti.reindex(columns = columns)
positive_children_not_on_hivhaiti.columns = ['_'.join(col) for col in positive_children_not_on_hivhaiti.columns.values]
positive_children_not_on_hivhaiti.fillna(0, inplace = True)
positive_children_not_on_hivhaiti.reset_index(inplace = True)

#========================================================================================================================
#Children unknown status
status_unknown = hhm_ff_stat[(~hhm_ff_stat.hiv_test_result.isin(['positif', 'negatif']))]

status_unknown.loc[(~status_unknown.hiv_test_result.isin(['positif', 'negatif'])), 'hiv_test_result'] = '---'

children_unknown_status = status_unknown.pivot_table(values = 'patient_code', index = 'Commune', columns = ['hiv_test_result', 'gender','age_range'],
                                                    aggfunc = len, fill_value = 0)
children_unknown_status.rename(columns = {'---': 'No_HIV_reported_Status_Unknown'}, inplace = True)

columns = [('No_HIV_reported_Status_Unknown','Female', '< 1'),
 ('No_HIV_reported_Status_Unknown','Female', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Female', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Female', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Female', '15 - 17'),
 ('No_HIV_reported_Status_Unknown','Male', '< 1'),
 ('No_HIV_reported_Status_Unknown','Male', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Male', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Male', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Male', '15 - 17')]
children_unknown_status = children_unknown_status.reindex(columns = columns)
children_unknown_status.columns = ['_'.join(col) for col in children_unknown_status.columns.values]
children_unknown_status.fillna(0, inplace = True)
children_unknown_status.reset_index(inplace = True)


#Sum
# children_unknown_status = children_unknown_status.merge(positive_children_not_on_hivhaiti, on = 'Commune', how = 'outer')
# children_unknown_status['No_HIV_reported_Status_Unknown'] = children_unknown_status[['No HIV reported (status Unknown_without positive not on Hiv Haiti)', 'No HIV reported (positive_not_on_hivhaiti)']].sum(axis = 1)
# children_unknown_status.drop(['No HIV reported (status Unknown_without positive not on Hiv Haiti)', 'No HIV reported (positive_not_on_hivhaiti)'], axis = 1, inplace = True)
# children_unknown_status



In [22]:
hhm_positive_children_not_on_hivhaiti.shape[0]
status_unknown.shape[0]

34

251

In [23]:
#SUM
children_status_unknown = pd.concat([positive_children_not_on_hivhaiti,children_unknown_status], ignore_index=True)
children_status_unknown = children_status_unknown.groupby(["Commune"]).sum()
children_status_unknown.reset_index(inplace = True)
children_status_unknown.head()
children_status_unknown.shape

,Commune,No_HIV_reported_Status_Unknown_Female_< 1,No_HIV_reported_Status_Unknown_Female_1 - 4,No_HIV_reported_Status_Unknown_Female_5 - 9,No_HIV_reported_Status_Unknown_Female_10 - 14,No_HIV_reported_Status_Unknown_Female_15 - 17,No_HIV_reported_Status_Unknown_Male_< 1,No_HIV_reported_Status_Unknown_Male_1 - 4,No_HIV_reported_Status_Unknown_Male_5 - 9,No_HIV_reported_Status_Unknown_Male_10 - 14,No_HIV_reported_Status_Unknown_Male_15 - 17
0,Anse D'Hainault,0.0,0,0,0,0,0,0,0,1,0
1,Anse-a-Galets,0.0,1,1,0,1,0,0,2,3,3
2,Aquin,0.0,0,0,1,0,0,0,0,0,1
3,Arcahaie,0.0,0,0,0,3,0,0,0,0,0
4,Cap-Haitien,0.0,0,0,1,0,0,0,1,0,0


(37, 11)

In [87]:
# children_status_unknown.to_excel("Verification_status_unk_10112022.xlsx", index = False)

In [24]:
#Concatenation _ FINAL DATA _ OVC_HIVSTAT_CHILDREN

# index = ['Commune','positive', 'on_arv', 'not_on_arv', 'negative', 'test not indicated', 'No_HIV_reported_Status_Unknown']
#------------------------------------------------------------------------------------------------------------------------
def ovc_hivstat_children():
    if len(positive_children) != 0 and len(children_on_arv) != 0 and len(children_not_on_arv) != 0 and len(negative_children) != 0 and len(children_status_unknown) != 0:
        ovc_hivstat_children = positive_children.merge(children_on_arv, on = 'Commune', how = 'outer').merge(children_not_on_arv, on = 'Commune', how = 'outer').merge(negative_children, on = 'Commune', how = 'outer').merge(children_status_unknown, on = 'Commune', how = 'outer')
        ovc_hivstat_children = ovc_hivstat_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_children['test not indicated'] = 0
        ovc_hivstat_children.fillna(0, inplace = True)
#         ovc_hivstat_children = ovc_hivstat_children.reindex(columns = index)        
        return ovc_hivstat_children
    elif len(positive_children) != 0 and len(children_on_arv) == 0 and len(children_not_on_arv) != 0 and len(negative_children) != 0 and len(children_status_unknown) != 0:
        ovc_hivstat_children = positive_children.merge(children_not_on_arv, on = 'Commune', how = 'outer').merge(negative_children, on = 'Commune', how = 'outer').merge(children_status_unknown, on = 'Commune', how = 'outer')
        ovc_hivstat_children = ovc_hivstat_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_children['children_on_arv','test not indicated'] = 0
        ovc_hivstat_children.fillna(0, inplace = True)
#         ovc_hivstat_children = ovc_hivstat_children.reindex(columns = index)        
        return ovc_hivstat_children
    elif len(positive_children) != 0 and len(children_on_arv) != 0 and len(children_not_on_arv) == 0 and len(negative_children) != 0 and len(children_status_unknown) != 0:
        ovc_hivstat_children = positive_children.merge(children_on_arv, on = 'Commune', how = 'outer').merge(negative_children, on = 'Commune', how = 'outer').merge(children_status_unknown, on = 'Commune', how = 'outer')
        ovc_hivstat_children = ovc_hivstat_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_children['children_not_on_arv','test not indicated'] = 0
        ovc_hivstat_children.fillna(0, inplace = True)
#         ovc_hivstat_children = ovc_hivstat_children.reindex(columns = index)        
        return ovc_hivstat_children
    elif len(positive_children) == 0 and len(negative_children) != 0 and len(children_status_unknown) != 0:
        ovc_hivstat_children = negative_children.merge(children_status_unknown, on = 'Commune', how = 'outer')
        ovc_hivstat_children = ovc_hivstat_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_children[['positive','on arv', 'not on arv','test not indicated']] = 0
        ovc_hivstat_children.fillna(0, inplace = True)
#         ovc_hivstat_children = ovc_hivstat_children.reindex(columns = index)
        return ovc_hivstat_children
    elif len(positive_children) == 0 and len(negative_children) == 0 and len(children_status_unknown) != 0:
        ovc_hivstat_children = children_status_unknown
        ovc_hivstat_children = ovc_hivstat_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_children[['positive','on arv', 'not on arv', 'negative','test not indicated']] = 0
        ovc_hivstat_children.fillna(0, inplace = True)
#         ovc_hivstat_children = ovc_hivstat_children.reindex(columns = index)
        return ovc_hivstat_children
    else:
        return 'HIV test data are not available right now!!'
    
    
#Calling the function
ovc_hivstat_children()

,Commune,positive_Female_< 1,positive_Female_1 - 4,positive_Female_5 - 9,positive_Female_10 - 14,positive_Female_15 - 17,positive_Male_< 1,positive_Male_1 - 4,positive_Male_5 - 9,positive_Male_10 - 14,...,No_HIV_reported_Status_Unknown_Female_1 - 4,No_HIV_reported_Status_Unknown_Female_5 - 9,No_HIV_reported_Status_Unknown_Female_10 - 14,No_HIV_reported_Status_Unknown_Female_15 - 17,No_HIV_reported_Status_Unknown_Male_< 1,No_HIV_reported_Status_Unknown_Male_1 - 4,No_HIV_reported_Status_Unknown_Male_5 - 9,No_HIV_reported_Status_Unknown_Male_10 - 14,No_HIV_reported_Status_Unknown_Male_15 - 17,test not indicated
0,Anse D'Hainault,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
1,Anse-a-Foleur,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,Anse-a-Galets,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,2.0,3.0,3.0,0
3,Anse-a-Veau,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,Aquin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0
5,Arcahaie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0
6,Cap-Haitien,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0
7,Capotille,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
8,Carrefour,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
9,Cavaillon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


# DATIM Other kids

In [25]:
#Creating Datim table for the other children
index_other = hhm_other.pivot_table(values = 'patient_code', index = 'Commune', columns = ['gender', 'age_range'], 
                                    aggfunc = len, fill_value = 0, margins = True, margins_name = 'Total')

columns = [('Female', '< 1'),
 ('Female', '1 - 4'),
 ('Female', '5 - 9'),
 ('Female', '10 - 14'),
 ('Female', '15 - 17'),
 ('Female', '18+'),
 ('Male', '< 1'),
 ('Male', '1 - 4'),
 ('Male', '5 - 9'),
 ('Male', '10 - 14'),
 ('Male', '15 - 17'),
 ('Male', '18+')]

index_other = index_other.reindex(columns = columns)
index_other[('Female', '18+')] = 0
index_other[('Male', '18+')] = 0
index_other.reset_index(inplace = True)

index_other

gender                 Commune Female                                 Male  \
age_range                         < 1 1 - 4 5 - 9 10 - 14 15 - 17 18+  < 1   
0              Anse D'Hainault      0     0     2       0       2   0    0   
1                Anse-a-Foleur      0     0     0       0       0   0    0   
2                Anse-a-Galets      0     0     0       0       0   0    0   
3                  Anse-a-Veau      0     0     0       2       0   0    0   
4                        Aquin      0     0     0       2       1   0    0   
5                     Arcahaie      0     3     0       5       3   0    0   
6                  Cap-Haitien      0     0     0       0       0   0    0   
7                    Capotille      0     0     0       0       0   0    0   
8                    Carrefour      1     2     7       2       2   0    0   
9                    Cavaillon      3     1     1       1       0   0    1   
10                       Cayes      0     0     0       0       0   0    0   
11          Croix-des-Bouquets      1     1     1       2       2   0    0   
12                      Delmas      1     3    28      40      26   0    0   
13                  Dessalines      1     0     1       0       0   0    1   
14                      Ennery      0     1     0       0       0   0    0   
15                    Gonaives      0     0     0       0       2   0    0   
16                  Gros-Morne      3     5     4       2       2   0    5   
17                      Hinche      0     1     0       0       0   0    0   
18                  Jean-Rabel      0     0     0       0       0   0    0   
19                     Jeremie      1     0     0       0       1   0    0   
20                    Kenscoff      0     0     1       2       0   0    1   
21                     L'Asile      1     0     0       0       0   0    0   
22                   La Tortue      0     0     1       0       0   0    0   
23                     Leogane      0     2     1       3       0   0    0   
24                       Limbe      0     3     5       7       6   0    0   
25                    Limonade      0     0     0       0       0   0    0   
26                   Marmelade      0     1     0       2       3   0    0   
27                       Milot      1     3     5       4       1   0    0   
28                   Miragoane      0     1     4       2       4   0    1   
29               Mombin-Crochu      0     0     0       0       0   0    0   
30               Mont-Organise      0     2     2       0       0   0    0   
31                 Ouanaminthe      1     1     1       0       1   0    0   
32                 Petit-Goave      1     2     1       7       1   0    1   
33                  Petit-Trou      0     0     0       0       0   0    0   
34                      Pignon      1     5     3      10       2   0    1   
35                      Pilate      2     5    10      14       4   0    1   
36              Plaine-du-Nord      0     1     1       0       0   0    0   
37                   Plaisance      0     0     0       0       0   0    0   
38                 Port-Margot      0     0     4       3       1   0    1   
39              Port-au-Prince      1     1     3       5       5   0    0   
40                Port-de-Paix      0     4     9       3       3   0    0   
41         Saint-Louis-du-Nord      0     0     0       0       0   0    0   
42                  Saint-Marc      0     0     1       1       1   0    0   
43               Saint-Raphael      0     1     0       0       0   0    0   
44                     Tabarre      0     3    10       7       5   0    0   
45               Terrier-Rouge      3     2     1       2       0   0    3   
46                Trou-du-Nord      1     1     2       3       2   0    0   
47                       Total     23    55   109     131      80   0   16   

gender                                     
age_range 1 - 4 5 - 9 10 - 14 15 - 17 18+  
0          

In [26]:
#Negative other children
hhm_negative_other_children = hhm_other_stat[hhm_other_stat.hiv_test_result == 'negatif']

negative_other_children = hhm_negative_other_children.pivot_table(index = 'Commune', values = 'patient_code', 
                                                      columns = ['hiv_test_result', 'gender','age_range'], aggfunc = len, fill_value = 0)
negative_other_children.rename(columns = {'negatif': 'negative'}, inplace = True)

columns = [('negative','Female', '< 1'),
 ('negative','Female', '1 - 4'),
 ('negative','Female', '5 - 9'),
 ('negative','Female', '10 - 14'),
 ('negative','Female', '15 - 17'),
 ('negative','Male', '< 1'),
 ('negative','Male', '1 - 4'),
 ('negative','Male', '5 - 9'),
 ('negative','Male', '10 - 14'),
 ('negative','Male', '15 - 17')]

negative_other_children = negative_other_children.reindex(columns = columns)
negative_other_children.columns = ['_'.join(col) for col in negative_other_children.columns.values]
negative_other_children.reset_index(inplace = True)

negative_other_children.head()

,Commune,negative_Female_< 1,negative_Female_1 - 4,negative_Female_5 - 9,negative_Female_10 - 14,negative_Female_15 - 17,negative_Male_< 1,negative_Male_1 - 4,negative_Male_5 - 9,negative_Male_10 - 14,negative_Male_15 - 17
0,Anse D'Hainault,0,0,1,0,0,0,0,2,0,0
1,Anse-a-Galets,0,0,0,0,0,0,1,3,0,0
2,Anse-a-Veau,0,0,0,1,0,0,0,2,1,0
3,Aquin,0,0,0,1,1,0,0,0,0,0
4,Arcahaie,0,3,0,3,1,0,1,1,1,2


In [27]:
hhm_negative_other_children.shape[0]

294

In [28]:
#Positive other children
# & (closed == 'FALSE')
hhm_other_stat.rename(columns = {'patient_code2':'full_code_patient_menage'}, inplace = True)
hhm_positive_other_children = hhm_other_stat[(hhm_other_stat.hiv_test_result == 'positif') & (hhm_other_stat.on_HIVHAITI == 'yes')].drop_duplicates('full_code_patient_menage')
hhm_positive_other_children = hhm_positive_other_children.merge(regime[['full_code_patient_menage', 'arv_start_date']], 
                                                    on = 'full_code_patient_menage', how = 'left')
hhm_positive_other_children['on_arv'] = hhm_positive_other_children.arv_start_date.map(on_arv)
positive_other_children = hhm_positive_other_children.pivot_table(values = 'patient_code', index = 'Commune', 
                                                      columns = ['hiv_test_result', 'gender','age_range'], aggfunc = len, fill_value = 0)
positive_other_children.rename(columns = {'positif': 'positive'}, inplace = True)

columns = [('positive','Female', '< 1'),
 ('positive','Female', '1 - 4'),
 ('positive','Female', '5 - 9'),
 ('positive','Female', '10 - 14'),
 ('positive','Female', '15 - 17'),
 ('positive','Male', '< 1'),
 ('positive','Male', '1 - 4'),
 ('positive','Male', '5 - 9'),
 ('positive','Male', '10 - 14'),
 ('positive','Male', '15 - 17')]

positive_other_children = positive_other_children.reindex(columns = columns)
positive_other_children.fillna(0, inplace = True)
positive_other_children.columns = ['_'.join(col) for col in positive_other_children.columns.values]
positive_other_children.reset_index(inplace = True)
positive_other_children

,Commune,positive_Female_< 1,positive_Female_1 - 4,positive_Female_5 - 9,positive_Female_10 - 14,positive_Female_15 - 17,positive_Male_< 1,positive_Male_1 - 4,positive_Male_5 - 9,positive_Male_10 - 14,positive_Male_15 - 17
0,Delmas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0


In [29]:
hhm_positive_other_children.shape[0]

1

In [30]:
#Dataset children on arv
hhm_other_children_on_arv = hhm_positive_other_children[hhm_positive_other_children.on_arv == 'yes']

other_children_on_arv = hhm_other_children_on_arv.pivot_table(values = 'patient_code', index = 'Commune', 
                                                  columns = ['on_arv', 'gender','age_range'], aggfunc = len, fill_value = 0)
other_children_on_arv.rename(columns = {'yes': 'on_arv'}, inplace = True)

columns = [('on_arv','Female', '< 1'),
 ('on_arv','Female', '1 - 4'),
 ('on_arv','Female', '5 - 9'),
 ('on_arv','Female', '10 - 14'),
 ('on_arv','Female', '15 - 17'),
 ('on_arv','Male', '< 1'),
 ('on_arv','Male', '1 - 4'),
 ('on_arv','Male', '5 - 9'),
 ('on_arv','Male', '10 - 14'),
 ('on_arv','Male', '15 - 17')]
other_children_on_arv = other_children_on_arv.reindex(columns = columns)
other_children_on_arv.columns = ['_'.join(col) for col in other_children_on_arv.columns.values]
other_children_on_arv.reset_index(inplace = True)

other_children_on_arv

#Dataset children not on arv
hhm_other_children_not_on_arv = hhm_positive_other_children[hhm_positive_other_children.on_arv == 'no']

other_children_not_on_arv = hhm_other_children_not_on_arv.pivot_table(values = 'patient_code', index = 'Commune',
                                                         columns = ['on_arv', 'gender','age_range'], aggfunc = len, fill_value = 0)

other_children_not_on_arv.rename(columns = {'no': 'not_on_arv'}, inplace = True)

columns = [('not_on_arv','Female', '< 1'),
 ('not_on_arv','Female', '1 - 4'),
 ('not_on_arv','Female', '5 - 9'),
 ('not_on_arv','Female', '10 - 14'),
 ('not_on_arv','Female', '15 - 17'),
 ('not_on_arv','Male', '< 1'),
 ('not_on_arv','Male', '1 - 4'),
 ('not_on_arv','Male', '5 - 9'),
 ('not_on_arv','Male', '10 - 14'),
 ('not_on_arv','Male', '15 - 17')]
other_children_not_on_arv = other_children_not_on_arv.reindex(columns = columns)
other_children_not_on_arv.columns = ['_'.join(col) for col in other_children_not_on_arv.columns.values]
other_children_not_on_arv.reset_index(inplace = True)
other_children_not_on_arv

,Commune,on_arv_Female_< 1,on_arv_Female_1 - 4,on_arv_Female_5 - 9,on_arv_Female_10 - 14,on_arv_Female_15 - 17,on_arv_Male_< 1,on_arv_Male_1 - 4,on_arv_Male_5 - 9,on_arv_Male_10 - 14,on_arv_Male_15 - 17
0,Delmas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN


,Commune,not_on_arv_Female_< 1,not_on_arv_Female_1 - 4,not_on_arv_Female_5 - 9,not_on_arv_Female_10 - 14,not_on_arv_Female_15 - 17,not_on_arv_Male_< 1,not_on_arv_Male_1 - 4,not_on_arv_Male_5 - 9,not_on_arv_Male_10 - 14,not_on_arv_Male_15 - 17


In [31]:
#Positive other_children not on HIV Haiti
hhm_positive_other_children_not_on_hivhaiti = hhm_other_stat[(hhm_other_stat.hiv_test_result == 'positif') & (hhm_other_stat.on_HIVHAITI == 'no')]

positive_other_children_not_on_hivhaiti = hhm_positive_other_children_not_on_hivhaiti.pivot_table(values = 'patient_code', index = 'Commune',
                                                                                     columns = ['hiv_test_result', 'gender','age_range'], aggfunc = len,
                                                                                     fill_value = 0)
positive_other_children_not_on_hivhaiti.rename(columns = {'positif': 'No_HIV_reported_Status_Unknown'}, inplace = True)

columns = [('No_HIV_reported_Status_Unknown','Female', '< 1'),
 ('No_HIV_reported_Status_Unknown','Female', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Female', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Female', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Female', '15 - 17'),
 ('No_HIV_reported_Status_Unknown','Male', '< 1'),
 ('No_HIV_reported_Status_Unknown','Male', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Male', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Male', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Male', '15 - 17')]
positive_other_children_not_on_hivhaiti = positive_other_children_not_on_hivhaiti.reindex(columns = columns)
positive_other_children_not_on_hivhaiti.columns = ['_'.join(col) for col in positive_other_children_not_on_hivhaiti.columns.values]
positive_other_children_not_on_hivhaiti.fillna(0, inplace = True)
positive_other_children_not_on_hivhaiti.reset_index(inplace = True)

#other_children unknown status
other_status_unknown = hhm_other[(~hhm_other.hiv_test_result.isin(['positif', 'negatif'])) & (hhm_other.age_in_year <= 17)]

other_status_unknown.loc[(~other_status_unknown.hiv_test_result.isin(['positif', 'negatif'])), 'hiv_test_result'] = '---'
other_children_unknown_status = other_status_unknown.pivot_table(values = 'patient_code', index = 'Commune', columns = ['hiv_test_result', 'gender','age_range'],
                                                    aggfunc = len, fill_value = 0)
other_children_unknown_status.rename(columns = {'---': 'No_HIV_reported_Status_Unknown'}, inplace = True)

columns = [('No_HIV_reported_Status_Unknown','Female', '< 1'),
 ('No_HIV_reported_Status_Unknown','Female', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Female', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Female', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Female', '15 - 17'),
 ('No_HIV_reported_Status_Unknown','Male', '< 1'),
 ('No_HIV_reported_Status_Unknown','Male', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Male', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Male', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Male', '15 - 17')]
other_children_unknown_status = other_children_unknown_status.reindex(columns = columns)
other_children_unknown_status.columns = ['_'.join(col) for col in other_children_unknown_status.columns.values]
other_children_unknown_status.fillna(0, inplace = True)
other_children_unknown_status.reset_index(inplace = True)

#=========================================================================================================
positive_other_children_not_on_hivhaiti.head()

,Commune,No_HIV_reported_Status_Unknown_Female_< 1,No_HIV_reported_Status_Unknown_Female_1 - 4,No_HIV_reported_Status_Unknown_Female_5 - 9,No_HIV_reported_Status_Unknown_Female_10 - 14,No_HIV_reported_Status_Unknown_Female_15 - 17,No_HIV_reported_Status_Unknown_Male_< 1,No_HIV_reported_Status_Unknown_Male_1 - 4,No_HIV_reported_Status_Unknown_Male_5 - 9,No_HIV_reported_Status_Unknown_Male_10 - 14,No_HIV_reported_Status_Unknown_Male_15 - 17
0,Delmas,0,0,0.0,0,0.0,0.0,0.0,0,0,1
1,Gros-Morne,1,0,0.0,0,0.0,0.0,0.0,0,0,0
2,Limbe,0,1,0.0,0,0.0,0.0,0.0,0,0,1
3,Petit-Goave,1,0,0.0,1,0.0,0.0,0.0,0,0,0
4,Pignon,0,1,0.0,0,0.0,0.0,0.0,0,0,0


In [32]:
hhm_positive_other_children_not_on_hivhaiti.shape[0]
other_status_unknown.shape[0]

12

479

In [33]:
#SUM
other_unknown_status_children = pd.concat([positive_other_children_not_on_hivhaiti, other_children_unknown_status], ignore_index=True)
other_unknown_status_children = other_unknown_status_children.groupby(["Commune"]).sum()
other_unknown_status_children.reset_index(inplace = True)
other_unknown_status_children.head()
other_unknown_status_children.shape

,Commune,No_HIV_reported_Status_Unknown_Female_< 1,No_HIV_reported_Status_Unknown_Female_1 - 4,No_HIV_reported_Status_Unknown_Female_5 - 9,No_HIV_reported_Status_Unknown_Female_10 - 14,No_HIV_reported_Status_Unknown_Female_15 - 17,No_HIV_reported_Status_Unknown_Male_< 1,No_HIV_reported_Status_Unknown_Male_1 - 4,No_HIV_reported_Status_Unknown_Male_5 - 9,No_HIV_reported_Status_Unknown_Male_10 - 14,No_HIV_reported_Status_Unknown_Male_15 - 17
0,Anse D'Hainault,0,0,1.0,0,2.0,0.0,9.0,4,0,1
1,Anse-a-Veau,0,0,0.0,1,0.0,0.0,0.0,2,0,0
2,Aquin,0,0,0.0,1,0.0,0.0,0.0,2,0,1
3,Arcahaie,0,0,0.0,2,2.0,0.0,0.0,0,3,0
4,Carrefour,0,0,2.0,0,0.0,0.0,0.0,2,0,1


(38, 11)

In [34]:
# index2 = ['Commune','positive', 'on_arv', 'not_on_arv', 'negative', 'test not indicated', 'No HIV reported (Status Unknown)']

#--------------------------------------------------------------------------------------------------------------------
#Concatenation _ OVC_HIVSTAT_OTHER_CHILDREN

def ovc_hivstat_other_children():
    if len(positive_other_children) != 0 and len(other_children_on_arv) != 0 and len(other_children_not_on_arv) != 0 and len(negative_other_children) != 0 and len(other_unknown_status_children) != 0:
        ovc_hivstat_other_children = positive_other_children.merge(other_children_on_arv, on = 'Commune', how = 'outer').merge(other_children_not_on_arv, on = 'Commune', how = 'outer').merge(negative_other_children, on = 'Commune', how = 'outer').merge(other_unknown_status_children, on = 'Commune', how = 'outer')
        ovc_hivstat_other_children = ovc_hivstat_other_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_other_children['test not indicated'] = 0
        ovc_hivstat_other_children.fillna(0, inplace = True)
#         ovc_hivstat_other_children = ovc_hivstat_other_children.reindex(columns = index2)        
        return ovc_hivstat_other_children
    elif len(positive_other_children) != 0 and len(other_children_on_arv) == 0 and len(other_children_not_on_arv) != 0 and len(negative_other_children) != 0 and len(other_unknown_status_children) != 0:
        ovc_hivstat_other_children = positive_other_children.merge(other_children_not_on_arv, on = 'Commune', how = 'outer').merge(negative_other_children, on = 'Commune', how = 'outer').merge(other_unknown_status_children, on = 'Commune', how = 'outer')
        ovc_hivstat_other_children = ovc_hivstat_other_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_other_children['other_children_on_arv','test not indicated'] = 0
        ovc_hivstat_other_children.fillna(0, inplace = True)
#         ovc_hivstat_other_children = ovc_hivstat_other_children.reindex(columns = index2)        
        return ovc_hivstat_other_children
    elif len(positive_other_children) != 0 and len(other_children_on_arv) != 0 and len(other_children_not_on_arv) == 0 and len(negative_other_children) != 0 and len(other_unknown_status_children) != 0:
        ovc_hivstat_other_children = positive_other_children.merge(other_children_on_arv, on = 'Commune', how = 'outer').merge(negative_other_children, on = 'Commune', how = 'outer').merge(other_unknown_status_children, on = 'Commune', how = 'outer')
        ovc_hivstat_other_children = ovc_hivstat_other_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_other_children['other_children_not_on_arv','test not indicated'] = 0
        ovc_hivstat_other_children.fillna(0, inplace = True)
#         ovc_hivstat_other_children = ovc_hivstat_other_children.reindex(columns = index2)        
        return ovc_hivstat_other_children
    elif len(positive_other_children) == 0 and len(negative_other_children) != 0 and len(other_unknown_status_children) != 0:
        ovc_hivstat_other_children = negative_other_children.merge(other_unknown_status_children, on = 'Commune', how = 'outer')
        ovc_hivstat_other_children = ovc_hivstat_other_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_other_children[['positive','on arv', 'not on arv','test not indicated']] = 0
        ovc_hivstat_other_children.fillna(0, inplace = True)
#         ovc_hivstat_other_children = ovc_hivstat_other_children.reindex(columns = index2)
        return ovc_hivstat_other_children
    elif len(positive_other_children) == 0 and len(negative_other_children) == 0 and len(other_unknown_status_children) != 0:
        ovc_hivstat_other_children = other_unknown_status_children
        ovc_hivstat_other_children = ovc_hivstat_other_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_other_children[['positive','on arv', 'not on arv', 'negative','test not indicated']] = 0
        ovc_hivstat_other_children.fillna(0, inplace = True)
#         ovc_hivstat_other_children = ovc_hivstat_other_children.reindex(columns = index2)
        return ovc_hivstat_other_children
    else:
        return 'HIV test data are not available right now!!'


#Calling the function
ovc_hivstat_other_children()
        

,Commune,positive_Female_< 1,positive_Female_1 - 4,positive_Female_5 - 9,positive_Female_10 - 14,positive_Female_15 - 17,positive_Male_< 1,positive_Male_1 - 4,positive_Male_5 - 9,positive_Male_10 - 14,...,No_HIV_reported_Status_Unknown_Female_1 - 4,No_HIV_reported_Status_Unknown_Female_5 - 9,No_HIV_reported_Status_Unknown_Female_10 - 14,No_HIV_reported_Status_Unknown_Female_15 - 17,No_HIV_reported_Status_Unknown_Male_< 1,No_HIV_reported_Status_Unknown_Male_1 - 4,No_HIV_reported_Status_Unknown_Male_5 - 9,No_HIV_reported_Status_Unknown_Male_10 - 14,No_HIV_reported_Status_Unknown_Male_15 - 17,"(other_children_not_on_arv, test not indicated)"
0,Anse D'Hainault,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,2.0,0.0,9.0,4.0,0.0,1.0,0
1,Anse-a-Galets,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,Anse-a-Veau,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0
3,Aquin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,0
4,Arcahaie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,2.0,0.0,0.0,0.0,3.0,0.0,0
5,Cap-Haitien,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
6,Carrefour,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0
7,Cavaillon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0
8,Cayes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0
9,Croix-des-Bouquets,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,0.0,0


# Caregivers

In [35]:
hh_caregiver = hhm_club[(hhm_club['caregiver_yes_no'] == str(1)) & (hhm_club['age_in_year'] >= 18)]
hh_caregiver.loc[(hh_caregiver.caregiver_yes_no == str(1)), 'caregiver_yes_no'] = 'yes'
hh_caregiver = hh_caregiver.drop_duplicates('patient_code').reset_index(drop = True)
print(f"Number of caregivers: {hh_caregiver.shape[0]}")

Number of caregivers: 813


In [36]:
hh_caregiver.to_excel('Caregiver_OEV_FY22Q4.xlsx', index = False)

In [37]:
caregiver = hh_caregiver.pivot_table(index = 'Commune', values = 'patient_code', columns = ['gender', 'caregiver_yes_no'], 
                                    aggfunc = len, fill_value = 0, margins = True, margins_name = 'Total')
caregiver.drop('Total', inplace = True)
caregiver.rename(columns = {'yes': 'Caregiver_18+'}, inplace = True)
caregiver.reset_index(inplace = True)
caregiver

gender,Commune,Female,Male,Total
caregiver_yes_no,,Caregiver_18+,Caregiver_18+,
0,Anse D'Hainault,6,2,8
1,Anse-a-Foleur,1,0,1
2,Anse-a-Galets,11,5,16
3,Anse-a-Veau,3,0,3
4,Aquin,9,6,15
5,Arcahaie,5,1,6
6,Cap-Haitien,6,3,9
7,Capotille,2,0,2
8,Carrefour,15,12,27


In [38]:
caregiver['Total'].sum()

813

# Export

In [39]:
import openpyxl

with pd.ExcelWriter('DATIM_OEV_FY22Q4_10112022.xlsx', engine = 'openpyxl') as writer:
    index_ff.to_excel(writer, sheet_name = 'siblings')
    ovc_hivstat_children().to_excel(writer, sheet_name = 'HIV_STAT_Direct')
    index_other.to_excel(writer, sheet_name = 'Index_other')
    ovc_hivstat_other_children().to_excel(writer, sheet_name = 'HIV_STAT_Other')
    caregiver.to_excel(writer, sheet_name = 'Caregivers_18+')

## Closing the connections...

In [5]:
driver.quit()
engine.dispose()

                                                                                                            The end